# Análise de Jogos da NFL com Regressão Logística

Este notebook utiliza a regressão logística para prever o vencedor de jogos da NFL com base em vitórias e derrotas acumuladas dos times. A seguir, apresentamos o passo a passo do código:

---

## 1. Importação de Bibliotecas

Importamos as bibliotecas necessárias para manipulação de dados, divisão de conjuntos, criação do modelo e avaliação de desempenho.

## 2. Carregamento do dataset

Utilizei o datset "games.csv", que contém dados de jogos da NFL de 1999 até os jogos atuais, o dataset é atualizado toda semana e eu atualizo através desse repositório: "https://github.com/nflverse/nfldata/blob/master/data/games.csv"

## 3. Filtragem dos dados
Filtrei apenas os jogos de temporada regular, de 2013 pra frente

## 4. Criação de features

Vitória e derrotas acumuladas no time de casa e fora

## 5. Divisão em treino e teste

Utilizei 80% dos dados para treino e 20% para teste

## 6. Treinamento do modelo

Criei um moelo de regressão logística, que visa predizer o vencedor da partida

## 7. Avaliação do modelo

Calcula a acurácia do modelo

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Carregar dataset
df = pd.read_csv('games.csv')

# Filtrar temporadas a partir de 2013 e jogos da temporada regular
df = df[df['season'] >= 2013]
df = df[df['game_type'] == 'REG']

# Definir o vencedor (1 se o time da casa vencer, 0 se o time visitante vencer)
df['winner'] = df['result'].apply(lambda x: 1 if x > 0 else 0)
df = df[df['result'] != 0]

# Contar vitórias do time da casa
df['home_team_wins'] = df.groupby(['home_team', 'season'])['winner'].cumsum()

# Contar vitórias do time visitante (inverso das vitórias do time da casa)
df['away_team_wins'] = df.groupby(['away_team', 'season'])['winner'].transform(lambda x: (1 - x).cumsum())

# Contar derrotas do time da casa (inverso das vitórias do time da casa)
df['home_team_losses'] = df.groupby(['home_team', 'season'])['winner'].transform(lambda x: (1 - x).cumsum())

# Contar derrotas do time visitante
df['away_team_losses'] = df.groupby(['away_team', 'season'])['winner'].cumsum()

# Selecionar as colunas de interesse para o modelo
parametros = ['home_team_wins', 'away_team_wins', 'home_team_losses', 'away_team_losses']

# Dividir em treino e teste
df_train = df[df['season'] < 2023]
df_test = df[df['season'] >= 2023]

X_train = df_train[parametros]
y_train = df_train['winner']
X_test = df_test[parametros]
y_test = df_test['winner']

# Criar o modelo de regressão logística
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# Fazer previsões
y_pred = modelo.predict(X_test)

# Avaliar a acurácia do modelo
acuracia = accuracy_score(y_test, y_pred)
print(f'Acurácia: {acuracia * 100:.2f}%')


Acurácia: 85.48%


In [2]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(modelo, X_train, y_train, cv=5)  # 5-fold cross-validation
print(f'Acurácia média com cross-validation: {scores.mean() * 100:.2f}%')


Acurácia média com cross-validation: 84.38%


In [3]:
# Variável para o número da semana que você deseja visualizar
semana = int(input("Digite o número da semana (por exemplo, 1 para semana 1): "))

# Filtrar os jogos da temporada de 2024 para a semana desejada
df_semana = df_test[(df_test['season'] == 2024) & (df_test['week'] == semana)].copy()

# Fazer previsões
y_pred_semana = modelo.predict(df_semana[parametros])

# Adicionar a coluna de vencedor previsto
df_semana['predicted_winner'] = y_pred_semana

# Função para exibir a predição com os nomes dos times
def display_prediction(row):
    if row['predicted_winner'] == 1:
        return f"{row['home_team']} vs {row['away_team']}, vencedor: {row['home_team']}"
    else:
        return f"{row['home_team']} vs {row['away_team']}, vencedor: {row['away_team']}"

# Aplicar a função no DataFrame de predições
df_semana['prediction_display'] = df_semana.apply(display_prediction, axis=1)

# Exibir as predições para todos os jogos da semana escolhida
for pred in df_semana['prediction_display']:
    print(pred)


Digite o número da semana (por exemplo, 1 para semana 1):  13


DET vs CHI, vencedor: DET
DAL vs NYG, vencedor: NYG
GB vs MIA, vencedor: MIA
KC vs LV, vencedor: KC
ATL vs LAC, vencedor: LAC
CIN vs PIT, vencedor: PIT
JAX vs HOU, vencedor: HOU
MIN vs ARI, vencedor: ARI
NE vs IND, vencedor: IND
NYJ vs SEA, vencedor: SEA
WAS vs TEN, vencedor: WAS
CAR vs TB, vencedor: TB
NO vs LA, vencedor: LA
BAL vs PHI, vencedor: PHI
BUF vs SF, vencedor: BUF
DEN vs CLE, vencedor: DEN


In [4]:
# Função para calcular lucro com base nas odds e predições
def calcular_lucro(row):
    aposta = 100  # Valor fixo da aposta em cada jogo
    if row['predicted_winner'] == 1:  # Aposta no time da casa
        odds = row['home_moneyline']
        if row['winner'] == 1:  # Aposta correta
            return aposta * (odds / 100) if odds > 0 else aposta * (100 / abs(odds))
        else:  # Aposta errada
            return -aposta
    else:  # Aposta no time visitante
        odds = row['away_moneyline']
        if row['winner'] == 0:  # Aposta correta
            return aposta * (odds / 100) if odds > 0 else aposta * (100 / abs(odds))
        else:  # Aposta errada
            return -aposta

# Adicionar colunas de odds no DataFrame de teste
df_test['predicted_winner'] = y_pred
df_test['lucro'] = df_test.apply(calcular_lucro, axis=1)

# Calcular lucro total e lucro médio por jogo
lucro_total = df_test['lucro'].sum()
lucro_medio = df_test['lucro'].mean()

print(f"Lucro total: ${lucro_total:.2f}")
print(f"Lucro médio por jogo: ${lucro_medio:.2f}")


Lucro total: $23936.46
Lucro médio por jogo: $50.61


C:\Users\guepp\AppData\Local\Temp\ipykernel_9008\1411890425.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predicted_winner'] = y_pred
C:\Users\guepp\AppData\Local\Temp\ipykernel_9008\1411890425.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['lucro'] = df_test.apply(calcular_lucro, axis=1)


In [5]:
# Função para calcular lucro com base nas odds e predições
def calcular_lucro(row):
    aposta = 100  # Valor fixo da aposta em cada jogo
    if row['predicted_winner'] == 1:  # Aposta no time da casa
        odds = row['home_moneyline']
        if row['winner'] == 1:  # Aposta correta
            return aposta * (odds / 100) if odds > 0 else aposta * (100 / abs(odds))
        else:  # Aposta errada
            return -aposta
    else:  # Aposta no time visitante
        odds = row['away_moneyline']
        if row['winner'] == 0:  # Aposta correta
            return aposta * (odds / 100) if odds > 0 else aposta * (100 / abs(odds))
        else:  # Aposta errada
            return -aposta

# Adicionar colunas de odds no DataFrame de teste
df_test['predicted_winner'] = y_pred
df_test['lucro'] = df_test.apply(calcular_lucro, axis=1)

# Calcular lucro total e lucro médio por jogo
lucro_total = df_test['lucro'].sum()
lucro_medio = df_test['lucro'].mean()

print(f"Lucro total: ${lucro_total:.2f}")
print(f"Lucro médio por jogo: ${lucro_medio:.2f}")

# Calcular lucro por semana
lucro_por_semana = df_test.groupby('week')['lucro'].sum()

# Adicionar porcentagem de lucro baseado em apostas totais por semana
apostas_por_semana = df_test.groupby('week').size() * 100  # Cada aposta é de $100
porcentagem_lucro_por_semana = (lucro_por_semana / apostas_por_semana) * 100

# Exibir resultados
for semana, lucro in lucro_por_semana.items():
    print(f"Semana {semana}:")
    print(f"  Lucro total: ${lucro:.2f}")
    print(f"  Porcentagem de lucro: {porcentagem_lucro_por_semana[semana]:.2f}%")


Lucro total: $23936.46
Lucro médio por jogo: $50.61
Semana 1:
  Lucro total: $2807.04
  Porcentagem de lucro: 87.72%
Semana 2:
  Lucro total: $3448.97
  Porcentagem de lucro: 107.78%
Semana 3:
  Lucro total: $3358.62
  Porcentagem de lucro: 104.96%
Semana 4:
  Lucro total: $2178.62
  Porcentagem de lucro: 68.08%
Semana 5:
  Lucro total: $2051.16
  Porcentagem de lucro: 73.26%
Semana 6:
  Lucro total: $1088.03
  Porcentagem de lucro: 37.52%
Semana 7:
  Lucro total: $805.00
  Porcentagem de lucro: 28.75%
Semana 8:
  Lucro total: $1482.78
  Porcentagem de lucro: 46.34%
Semana 9:
  Lucro total: $1682.16
  Porcentagem de lucro: 58.01%
Semana 10:
  Lucro total: $326.19
  Porcentagem de lucro: 11.65%
Semana 11:
  Lucro total: $1290.81
  Porcentagem de lucro: 46.10%
Semana 12:
  Lucro total: $1603.87
  Porcentagem de lucro: 55.31%
Semana 13:
  Lucro total: $1043.63
  Porcentagem de lucro: 35.99%
Semana 14:
  Lucro total: $-478.32
  Porcentagem de lucro: -17.08%
Semana 15:
  Lucro total: $248.0

C:\Users\guepp\AppData\Local\Temp\ipykernel_9008\3984357172.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predicted_winner'] = y_pred
C:\Users\guepp\AppData\Local\Temp\ipykernel_9008\3984357172.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['lucro'] = df_test.apply(calcular_lucro, axis=1)
